In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import bs4
from urllib.request import urlopen as uReq
from urllib.request import Request, urlopen
from unidecode import unidecode

import os
import re
import pickle
import json
import copy

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
binary = FirefoxBinary(r'C:\Program Files\Mozilla Firefox\firefox.exe')
driver = webdriver.Firefox(firefox_binary=binary)

Sherdog

In [4]:
fighters_sherdog = json.load(open('fighters_sherdog','r'))

In [63]:
def sherdog_date_decoder(string,short=True):
    if short:
        months = {'Jan': '01','Feb': '02','Mar': '03',
          'Apr': '04','May': '05','Jun': '06',
          'Jul': '07','Aug': '08','Sep': '09',
          'Oct': '10','Nov': '11','Dec': '12'}
        new_date = string.split('/')
        return new_date[2].strip()+'-'+months[new_date[0].strip()]+'-'+new_date[1].strip()
    else:
        months = {"January":"01","February":"02","March":"03","April":"04","May":"05","June":"06",
                  "July":"07","August":"08","September":"09","October":"10","November":"11","December":"12"}
        parts = string.split(' ')
        return f"{parts[2]}-{months[parts[0]]}-{parts[1].replace(',','')}"
        

In [173]:
def get_fighters_info_sherdog(url):
    fighter = {'Name':'',
               'Nickname':'',
               'Birth date':'',
               'Height':'',
               'Record':{'wins':'','loses':'','draws':'','nc':''},
               'Affiliation':'',
               'Nationality':'',
               'Location region':'',
               'Location city':'',
               'Weight':'',
               'wins summary': {'KO/TKO':'','SUBMISSIONS':'','DECISIONS':'','OTHERS':''},
               'loses summary': {'KO/TKO':'','SUBMISSIONS':'','DECISIONS':'','OTHERS':''},
               'bouts':[]}
    
    driver.get(url)
    site = BeautifulSoup(driver.page_source,'html.parser')
    
    #fighters summary
    bio = site.find('div',class_='module bio_fighter vcard')
    
    
    if not site.find('div',class_='module bio_fighter tagged'):
        if len(bio.find('h1').find_all('span')) > 1:
            fighter['Name'],fighter['Nickname'] = [x.text for x in bio.find('h1').find_all('span')]
        else:
            fighter['Name'] = str(bio.find('h1').find('span').text)
    else:
        fighter['Name'] = str(site.find('span',class_='fn').text)
        fighter['Nickname'] = str(site.find('span',class_='nickname').text if site.find('span',class_='nickname') else '')
        
    
    fighter['Birth date'] = str(bio.find('span',{'itemprop':'birthDate'}).text if bio.find('span',{'itemprop':'birthDate'}) else '')
    if re.search('(\d+\.\d+) cm',bio.find('div',class_='size_info').find('span',class_='item height').text):
        fighter['Height'] = str(re.search('(\d+\.\d+) cm',bio.find('div',class_='size_info').find('span',class_='item height').text)[1])
    if re.search('(\d+\.\d+) kg',bio.find('div',class_='size_info').find('span',class_='item weight').text):
        fighter['Weight'] = str(re.search('(\d+\.\d+) kg',bio.find('div',class_='size_info').find('span',class_='item weight').text)[1])
    fighter['Record']['wins'] = str(bio.find('span',class_='counter').text)
    fighter['Record']['loses'] = str(bio.find('div',class_='bio_graph loser').find('span',class_='counter').text)
    
    if bio.find('div',class_='right_side'):
        for outs in bio.find('div',class_='right_side').find_all('div',class_='bio_graph'):
            if outs.find('span','result').text == 'Draws': fighter['Record']['draws'] = str(outs.find('span','counter').text)
            if outs.find('span','result').text == 'N/C': fighter['Record']['nc'] = str(outs.find('span','counter').text)
    
    bouts_summary = [re.match('\d+',x.text)[0] for x in bio.find_all('span',class_='graph_tag')]
    if len(bouts_summary) == 8:
        fighter['wins summary']['KO/TKO'], fighter['wins summary']['SUBMISSIONS'], fighter['wins summary']['DECISIONS'], fighter['wins summary']['OTHERS'] = bouts_summary[0:4]
        fighter['loses summary']['KO/TKO'], fighter['loses summary']['SUBMISSIONS'], fighter['loses summary']['DECISIONS'], fighter['loses summary']['OTHERS'] = bouts_summary[4:]
    else:
        fighter['wins summary']['KO/TKO'], fighter['wins summary']['SUBMISSIONS'], fighter['wins summary']['DECISIONS'] = bouts_summary[0:3]
        fighter['loses summary']['KO/TKO'], fighter['loses summary']['SUBMISSIONS'], fighter['loses summary']['DECISIONS'] = bouts_summary[3:]
        
    fighter['Nationality'] = str(bio.find('strong',{'itemprop':'nationality'}).text if bio.find('strong',{'itemprop':'nationality'}) else '')
    fighter['Affiliation'] =str(bio.find('a',class_='association').text if bio.find('a',class_='association') else '')
    
    if bio.find('span',class_='locality'):
        if ',' in bio.find('span',class_='locality').text:
            fighter['Location city'], fighter['Location region'] = bio.find('span',class_='locality').text.split(',')
        else:
            fighter['Location city'] = str(bio.find('span',class_='locality').text)
    
    #fights description
    for history in site.find_all('div',class_='module fight_history'):
        if (re.search('Pro Exhibition',history.find('h2').text,re.IGNORECASE) or 
        re.search('Amateur',history.find('h2').text,re.IGNORECASE) or
        re.search('Upcoming',history.find('h2').text,re.IGNORECASE)): continue

        for bout in history.find_all('tr')[1:][::-1]:
            bout_info = {'result':'','opponent':'','event':'','date':'','method':'','referee':'','round':'','time':'','dc reason':""}
            fight_info = bout.find_all('td')
            bout_info['result'] = str(fight_info[0].text)
            bout_info['opponent'] = str(fight_info[1].text)
            bout_info['event'] = str(fight_info[2].a.text)
            bout_info['date'] = str(sherdog_date_decoder(fight_info[2].find('span','sub_line').text) if fight_info[2].find('span','sub_line') else '')
            bout_info['method'] = str(fight_info[3].contents[0])
            if len(fight_info[3].contents) == 2:
                pass
            elif len(fight_info[3].contents[2]) == 1: bout_info['referee'] = str(fight_info[3].contents[2].text if fight_info[3].contents[2] else '')
            else:
                bout_info['referee'] = str(fight_info[3].contents[4].text if str(fight_info[3].contents[4]) != 'N/A' else '')
                bout_info['dc reason'] = str(unidecode(fight_info[3].contents[2]))

            bout_info['round'] = str(fight_info[4].text)
            bout_info['time'] = str(fight_info[5].text)
            fighter['bouts'].append(bout_info)
        
    return fighter

In [174]:
def update_sherdog(size=1):
    driver.get(f"https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/1")
    site =  BeautifulSoup(driver.page_source,'html.parser')
    fighters_links = []
    
    for card in site.find('div',{'id':'recent_tab'}).find('tbody').find_all('tr')[1:1+size]:
        link = "https://www.sherdog.com" + card.a['href']
        driver.get(link)
        site = BeautifulSoup(driver.page_source,'html.parser')
        for main_card in site.find('div',class_='module fight_card').find_all('a'):
            if main_card.find('span'):
                if main_card.find('span').text not in fighters_links:
                    fighters_links.append(get_fighters_info_sherdog("https://www.sherdog.com"+main_card['href']))

        for fight in site.find('div',class_='content table').find_all('tr',{'itemprop':'subEvent'}):
            for fighter in fight.find_all('td',{'itemprop':'performer'}):
                if fighter.a.text not in fighters_links:
                    fighters_links.append(get_fighters_info_sherdog("https://www.sherdog.com"+fighter.a['href']))
                
    return fighters_links

In [175]:
def update(old,new):
    to_delete = [y for y,x in enumerate(old) if x['Name'] in [x['Name'] for x in new]]
    for fighter_index in sorted(to_delete)[::-1]:
        del old[fighter_index]
    return old + new

In [176]:
sherdog_update_dbase = update_sherdog(2)

In [177]:
updated_list_sherdog = update(fighters_sherdog,sherdog_update_dbase)

In [178]:
len(updated_list_sherdog)

1934

In [179]:
json.dump(updated_list_sherdog,open('fighters_sherdog_June_3','w'))

Tapology

In [101]:
fighters = json.load(open('tapology_fighters_June_3.txt','r'))

In [147]:
link = urlopen(f'https://www.tapology.com/fightcenter?group=ufc&page=0&region=&schedule=results')
site = BeautifulSoup(link,'html.parser')

cards = [("https://www.tapology.com" + x.find('span','name').a['href']) for x in site.find_all('section',class_="fcListing")][0:2]
fighters_renew= []
fighters_updated = []

for link in cards:
    site = BeautifulSoup(urlopen(link),'html.parser')


    for bout in site.find_all('li','fightCard'):
            for name in bout.find_all('a'):
                if re.match('/fightcenter/bouts/',name.get('href')): continue
                fighters_renew.append("https://www.tapology.com" + name.get('href'))  

In [141]:
def get_last_ufc_card(size=1):
    link = urlopen(f'https://www.tapology.com/fightcenter?group=ufc&page=0&region=&schedule=results')
    site = BeautifulSoup(link,'html.parser')

    cards = [("https://www.tapology.com" + x.find('span','name').a['href']) for x in site.find_all('section',class_="fcListing")][0:size]
    fighters_renew= []
    fighters_updated = []
    
    for link in cards:
        site = BeautifulSoup(urlopen(link),'html.parser')


        for bout in site.find_all('li','fightCard'):
                for name in bout.find_all('a'):
                    if re.match('/fightcenter/bouts/',name.get('href')): continue
                    fighters_renew.append("https://www.tapology.com" + name.get('href'))  

    for fighter in list(set(fighters_renew)):
        fighters_updated.append(get_fighters_info(fighter))
    
    return fighters_updated

In [117]:
def get_score(string):
    if not string: return {'nc':'0','wins':'0', 'loses':'0','draws':'0'}
    score = {'nc':'0'}
    score['wins'], score['loses'], score['draws'] = re.search('\d+-\d+-\d+',string)[0].split('-')
    if re.search('NC',string):
        score['nc'] = re.search('(\d+) NC',"16-4-0, 1 NC (Win-Loss-Draw)")[1]
    return score

def get_last_fight_date(string):
    months = {'January': '01','February': '02','March': '03',
          'April': '04','May': '05','June': '06',
          'July': '07','August': '08','September': '09',
          'October': '10','November': '11','December': '12'}
    month = months[re.search('\w+',string)[0]]
    day, year = list(re.findall('\d+',string))
    return f'{year}-{month}-{day}'

In [172]:
def get_fighters_info(url):
    driver.implicitly_wait(2)
    driver.get(url)
    
    element = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.ID, "fighterRecordControls")))
    element.find_element_by_class_name('right').click()
#   driver.find_element_by_id('fighterRecordControls').find_element_by_class_name('right').click()
    site = BeautifulSoup(driver.page_source,'html.parser')
    
    fighter = {'Name':'',
       'Nickame':'',
       'Birth date':'',
       'Given name':'',
       'Height':'',
       'Reach':'',
       'Record':{'wins':'','loses':'','draws':'','nc':''},
       'Last fight':'',
       'Affiliation':'',
       'Born':'',
       'Fighting out of':'',
       'bouts':[]}
    

    fighter['Name'] = unidecode(site.find('div','fighterUpcomingHeader').find_all('h1')[1].string)
    fighter['Nickame'] = '' if not site.find('div','fighterUpcomingHeader').find('h4','nickname').string else site.find('div','fighterUpcomingHeader').find('h4','nickname').string.strip()[1:-1]

    for row in site.find('div','details details_two_columns').find_all('li'):
        for strong_text,plain_text in zip(row.find_all('strong'),row.find_all('span')):
            strong_text, plain_text = strong_text.string, plain_text.string
            if re.search('Given Name:',strong_text): fighter['Given name'] = plain_text
            if re.search('Pro MMA Record:',strong_text): fighter['Record'] = get_score(plain_text)
            if re.search('Date of Birth',strong_text): fighter['Birth date'] = plain_text.replace('.','-')
            if re.search('Last Fight:',strong_text): fighter['Last fight'] = get_last_fight_date(plain_text)
            if re.search('Affiliation:',strong_text): fighter['Affiliation'] = plain_text
            if re.search('Height:',strong_text) and re.search('\((.*)cm\)',plain_text): fighter['Height'] = re.search('\((.*)cm\)',plain_text)[1]
            if re.search('Reach:',strong_text)  and re.search('\((.*)cm\)',plain_text): fighter['Reach'] = re.search('\((.*)cm\)',plain_text)[1]
            if re.search('Born:',strong_text): fighter['Born'] = plain_text
            if re.search('Fighting out of:',strong_text): fighter['Fighting out of'] = plain_text
    
    for fight in site.find('div',id='fighterRecord').find_all('li')[::-1]:
        
        bout_info = {
            'Billing':'',
            'Class weight':'',
            'Decision':'',
            'Duration':'',
            'Fight under':'',
            "Fighter's record":'',
            'Injuries':'',
            'Odds':'',
            "Opponent's name":'',
            "Opponent's record":'',
            'Pay':'',
            'Referee':'',
            'Round':'',
            'Time':'',
            'Title in a fight':'',
            'Way':'',
            'Weigh-In':'',
            'Weightclass':'',
            'Weightclass title name':'',
            'Win or Lose':'',
            'fight date':''}
        
        
        if (fight.find('div','record nonMma') or 
            re.search('Cancelled Bout', fight.find('div','lead').text) or 
            re.search('Confirmed Upcoming Bout', fight.find('div','lead').text) or 
            fight.find('div',{'data-result':'cancelled'})): continue

        bout_info["Opponent's name"] = unidecode(fight.find('div','name').text)
        
        if fight.find('span',{"title":'Fighter Record Before Fight'}):
            bout_info["Fighter's record"] = get_score(fight.find('span',{"title":'Fighter Record Before Fight'}).text)
            del bout_info["Fighter's record"]['nc']
        else:
            bout_info["Fighter's record"] = {'wins':'0','loses':'0','draws':'0'}
            
        if fight.find('span',{"title":'Opponent Record Before Fight'}):
            bout_info["Opponent's record"] = get_score(fight.find('span',{"title":'Opponent Record Before Fight'}).text)
            del bout_info["Opponent's record"]['nc']

        if '·' in fight.find('div','lead').text:
            win_info = fight.find('div','lead').text.split('·')
            if len(win_info) == 2:
                bout_info["Win or Lose"] =  win_info[0]
                bout_info["Way"] =  win_info[1]                
            elif len(win_info) == 3:
                bout_info["Win or Lose"] =  win_info[0]
                bout_info["Way"] =  win_info[1]
                bout_info["Decision"] = win_info[2]
            else:
                bout_info["Win or Lose"] = win_info[0]
                bout_info["Way"] = win_info[1]
                bout_info["Time"] = win_info[2]
                bout_info['Round'] = win_info[3]
        else:
            bout_info["Win or Lose"] = fight.find('div','lead').text
        
        bout_info["Fight under"] = fight.find('div','notes').text
        bout_info["fight date"] = fight.find('div','date').text.replace('.','-')

        if fight.find('div',class_='detail tall'):
            for additional_info in fight.find('div',class_='detail tall').find_all('div'):
                if additional_info.find_all('span'):
                    first, second = list(additional_info.find_all('span'))

                    if first.text == 'Title Bout:': 
                        if '·' in second.text:
                            bout_info['Title in a fight'], bout_info['Weightclass title name'] = second.text.split('·')
                        else: 
                            bout_info['Weightclass title name'] = second.text
                    
                    if first.text == 'Billing:': bout_info['Billing'] = second.text
                    if first.text == 'Duration:':
                        if re.search("(.*\d)",second.text)[0]:
                            bout_info['Duration'] = re.search("(.*\d)",second.text)[0]
                        else:
                            bout_info['Duration'] = second.text
                        

                    if first.text == 'Weight:':
                        for weightclass_info in second.text.split('·'):
                            if not re.search("\d",weightclass_info):
                                bout_info['Weightclass'] = weightclass_info

                            elif re.search("Weigh-In",weightclass_info,re.IGNORECASE):
                                bout_info['Weigh-In'] = re.search('(\d+\.?\d? lbs)',weightclass_info)[0]
                                
                            else:
                                bout_info['Class weight'] = re.search('(\d+\.?\d? lbs)',weightclass_info)[0] if re.search('(\d+\.?\d? lbs)',weightclass_info) else ''


                    if first.text == 'Odds:': bout_info['Odds'] = second.text.split('·')[0]
                    if first.text == 'Referee:': bout_info['Referee'] = second.text
                    if first.text == 'Disclosed Pay:': bout_info['Pay'] = re.search('\$(\d+) ',second.text.replace(',',''))[1]
                    if first.text == 'Injuries:': bout_info['Injuries'] = second.text
        
        fighter['bouts'].append(bout_info)
    
    return fighter

In [119]:
def update(old,new):
    to_delete = [y for y,x in enumerate(old) if x['Name'] in [x['Name'] for x in new]]
    for fighter in sorted(to_delete)[::-1]:
        del old[fighter]
    return old + new

In [142]:
new_info = get_last_ufc_card(size=2)

In [152]:
updated_tapology_fighters = update(fighters,new_info)

In [154]:
json.dump(updated_tapology_fighters,open('tapology_fighters_June_3.txt','w'))

ufc stats

In [186]:
fighters_ufc_stats = json.load(open('all_ufc_stats_info_June_3','r'))

In [187]:
def get_all_fighters(cards_links,fighter_links = dict()):
    for link in cards_links:
        site = BeautifulSoup(urlopen(link),'html.parser')
        for fight in site.find('tbody',class_='b-fight-details__table-body').find_all('tr',class_='b-fight-details__table-row'):
            for fighter in fight.find_all('a',class_='b-link_style_black'):
                if fighter.text.strip() not in fighter_links:
                    fighter_links[fighter.text.strip()] = fighter['href']
    
    return fighter_links

In [198]:
link = urlopen(f'http://ufcstats.com/statistics/events/completed')
site = BeautifulSoup(link,'html.parser')
info = []
for card in site.find('tbody').find_all('a',class_='b-link')[1:3]:
    print(card['href'])

http://ufcstats.com/event-details/cbc071cb20ea59c7
http://ufcstats.com/event-details/6b8f28da9a483049


In [188]:
def get_last_card(size=1): 
    link = urlopen(f'http://ufcstats.com/statistics/events/completed')
    site = BeautifulSoup(link,'html.parser')
    info = []
    for card in site.find('tbody').find_all('a',class_='b-link')[1:size+1]:
        link = card['href']
        fighters = get_all_fighters([link])
    
        for fighter in fighters.keys():
            print(fighter)
            info.append(get_fighters_info(fighters[fighter]))
    
    return info

In [189]:
def get_bout_info(link):
    site = BeautifulSoup(urlopen(link),'html.parser')
    
    bout_info = {'Card name':'',
                 'Bout name':'',
                 "fight bonus":'', 
                 'perf bonus':'', 
                 'sub bonus':'',
                 'ko bonus':'', 
                 'Method':'',
                 'Round':'',
                 'title fight':'',
                 'Details':'',
                 'Score 1':'',
                 'Score 2':'',
                 'Score 3':'',
                 'Fighter1 stats':{
                     'Totals':{'Name':'',
                         'KD':'',
                               'SIG. STR.':{'attempted':'','landed':''},
                               'SIG. STR. %':'',
                               'TOTAL STR.':{'attempted':'','landed':''},
                               'TD':{'attempted':'','landed':''},
                               'TD %':'',
                               'SUB. ATT':'',
                               'PASS':'',
                               'REV.':''},

                     'Totals per Round':[],

                     'Significant Strikes':{'Name':'',
                         'SIG. STR.':{'attempted':'','landed':''},
                               'SIG. STR. %':'',
                               'HEAD':{'attempted':'','landed':''},
                               'BODY':{'attempted':'','landed':''},
                               'LEG':{'attempted':'','landed':''},
                               'DISTANCE':{'attempted':'','landed':''},
                               'CLINCH':{'attempted':'','landed':''},
                               'GROUND':{'attempted':'','landed':''}},

                     'Significant Strikes per Round':[],
                 },

                 'Fighter2 stats':{
                     'Totals':{'Name':'',
                               'KD':'',
                               'SIG. STR.':{'attempted':'','landed':''},
                               'SIG. STR. %':'',
                               'TOTAL STR.':{'attempted':'','landed':''},
                               'TD':{'attempted':'','landed':''},
                               'TD %':'',
                               'SUB. ATT':'',
                               'PASS':'',
                               'REV.':''},

                     'Totals per Round':[],

                     'Significant Strikes':{
                         'Name':'',      
                         'SIG. STR.':{'attempted':'','landed':''},
                               'SIG. STR. %':'',
                               'HEAD':{'attempted':'','landed':''},
                               'BODY':{'attempted':'','landed':''},
                               'LEG':{'attempted':'','landed':''},
                               'DISTANCE':{'attempted':'','landed':''},
                               'CLINCH':{'attempted':'','landed':''},
                               'GROUND':{'attempted':'','landed':''}},

                     'Significant Strikes per Round':[],
                 },

                'Fighter1':{'Result':'','Name':'', },
                'Fighter2':{'Result':'','Name':'', }}


    totals_round = {'Round':'',
                    'Name':'',
                    'KD':'',
                    'SIG. STR.':{'attempted':'','landed':''},
                    'SIG. STR. %':'',
                    'TOTAL STR.':{'attempted':'','landed':''},
                    'TD':{'attempted':'','landed':''},
                    'TD %':'',
                    'SUB. ATT':'',
                    'PASS':'',
                    'REV.':''}

    sigstrikes_round = {'Name':'','Round':'',
                       'SIG. STR.':{'attempted':'','landed':''},
                       'SIG. STR. %':'',
                       'HEAD':{'attempted':'','landed':''},
                       'BODY':{'attempted':'','landed':''},
                       'LEG':{'attempted':'','landed':''},
                       'DISTANCE':{'attempted':'','landed':''},
                       'CLINCH':{'attempted':'','landed':''},
                       'GROUND':{'attempted':'','landed':''}}
    
    #Card name
    bout_info['Card name'] = site.find('a',class_='b-link').text.strip()
    bout_info['Bout name'] = site.find('i',class_='b-fight-details__fight-title').text.strip()
    bout_info['Fighter1']['Result'], bout_info['Fighter2']['Result'] = [x.text.strip() for x in site.find('div',class_='b-fight-details__persons clearfix').find_all('i')]
    bout_info['Fighter1']['Name'], bout_info['Fighter2']['Name'] = [x.text.strip() for x in site.find('div',class_='b-fight-details__persons clearfix').find_all('h3')]
    bout_info['Fighter1']['Nickname'], bout_info['Fighter2']['Nickname'] = [x.text.strip()[1:-1] for x in site.find('div',class_='b-fight-details__persons clearfix').find_all('p')]
    for bonus in site.find('i',class_='b-fight-details__fight-title').find_all('img'):
        if re.search('/(\w+).png',bonus['src'])[1] == 'belt': bout_info['title fight'] = True
        else:
            bout_info[re.search('/(\w+).png',bonus['src'])[1]+' bonus'] = True
    #Methods
    method = site.find('p',class_="b-fight-details__text").find('i','b-fight-details__text-item_first').contents
    bout_info[method[1].text.strip()[:-1]] = method[3].text.strip()
    descs = site.find('p',class_="b-fight-details__text").find_all('i',class_="b-fight-details__text-item")
    bout_info[descs[0].contents[1].text.strip()[:-1]] = descs[0].contents[2].strip()
    bout_info[descs[1].contents[1].text.strip()[:-1]] = descs[1].contents[2].strip()
    bout_info[descs[2].contents[1].text.strip()[:-1]] = descs[2].contents[2].strip()
    bout_info[descs[3].contents[1].text.strip()[:-1]] = descs[3].contents[3].text.strip()

    #Refs or details
    if len(site.find_all('p',class_='b-fight-details__text')[1].find_all('i','b-fight-details__text-item')) == 3:
        cards = site.find_all('p',class_='b-fight-details__text')[1].find_all('i','b-fight-details__text-item')

        bout_info['Score 1'] = {'Name':cards[0].contents[1].text}
        bout_info['Score 1']['Fighter1'], bout_info['Score 1']['Fighter2'] = cards[0].contents[2].strip()[:-1].split('-')

        bout_info['Score 2'] = {'Name':cards[0].contents[1].text}
        bout_info['Score 2']['Fighter1'], bout_info['Score 2']['Fighter2'] = cards[1].contents[2].strip()[:-1].split('-')

        bout_info['Score 3'] = {'Name':cards[0].contents[1].text}
        bout_info['Score 3']['Fighter1'], bout_info['Score 3']['Fighter2'] = cards[2].contents[2].strip()[:-1].split('-')
    else:
        bout_info['Details'] = site.find_all('p',class_='b-fight-details__text')[1].contents[2].strip()
        
    if not site.find('section',class_='b-fight-details__section js-fight-section').find('p'):
        return bout_info
    
    stats = site.find_all('tbody',class_='b-fight-details__table-body')

    #Totals 
    f1,f2 = bout_info['Fighter1 stats']['Totals'], bout_info['Fighter2 stats']['Totals']
    totals_head = stats[0].find_all('td')
    f1['Name'],f2['Name'] = [x.text.strip() for x in totals_head[0].find_all('p')]
    f1['KD'],f2['KD'] = [x.text.strip() for x in totals_head[1].find_all('p')]
    f1['SIG. STR.']['landed'],f1['SIG. STR.']['attempted'], f2['SIG. STR.']['landed'],f2['SIG. STR.']['attempted'] = [a.strip() for x in totals_head[2].find_all('p') for a in x.text.split('of')]
    f1['SIG. STR. %'],f2['SIG. STR. %'] = [x.text.strip() for x in totals_head[3].find_all('p')]
    f1['TOTAL STR.']['landed'],f1['TOTAL STR.']['attempted'], f2['TOTAL STR.']['landed'],f2['TOTAL STR.']['attempted'] = [a.strip() for x in totals_head[4].find_all('p') for a in x.text.split('of')]
    f1['TD']['landed'],f1['TD']['attempted'], f2['TD']['landed'],f2['TD']['attempted'] = [a.strip() for x in totals_head[5].find_all('p') for a in x.text.split('of')]
    f1['TD %'],f2['TD %'] = [x.text.strip() for x in totals_head[6].find_all('p')]
    f1['SUB. ATT'],f2['SUB. ATT'] = [x.text.strip() for x in totals_head[7].find_all('p')]
    f1['PASS'],f2['PASS'] = [x.text.strip() for x in totals_head[8].find_all('p')]
    f1['REV.'],f2['REV.'] = [x.text.strip() for x in totals_head[9].find_all('p')]

    #TotalsPR
    totals_per_round = stats[1].find_all('td')

    for col_index in range(0,len(totals_per_round),10):
        fighter1round = copy.deepcopy(totals_round)
        fighter2round = copy.deepcopy(totals_round)

        fighter1round['Round'], fighter2round['Round'] = [f'{(col_index//10)+1}'] *2
        fighter1round['Name'],fighter2round['Name'] = [x.text.strip() for x in totals_per_round[0+col_index].find_all('p')]
        fighter1round['KD'],fighter2round['KD'] = [x.text.strip() for x in totals_per_round[1+col_index].find_all('p')]
        fighter1round['SIG. STR.']['landed'],fighter1round['SIG. STR.']['attempted'], fighter2round['SIG. STR.']['landed'],fighter2round['SIG. STR.']['attempted'] = [a.strip() for x in totals_per_round[2+col_index].find_all('p') for a in x.text.split('of')]
        fighter1round['SIG. STR. %'],fighter2round['SIG. STR. %'] = [x.text.strip() for x in totals_per_round[3+col_index].find_all('p')]
        fighter1round['TOTAL STR.']['landed'],fighter1round['TOTAL STR.']['attempted'], fighter2round['TOTAL STR.']['landed'],fighter2round['TOTAL STR.']['attempted'] = [a.strip() for x in totals_per_round[4+col_index].find_all('p') for a in x.text.split('of')]
        fighter1round['TD']['landed'],fighter1round['TD']['attempted'], fighter2round['TD']['landed'],fighter2round['TD']['attempted'] = [a.strip() for x in totals_per_round[5+col_index].find_all('p') for a in x.text.split('of')]
        fighter1round['TD %'],fighter2round['TD %'] = [x.text.strip() for x in totals_per_round[6+col_index].find_all('p')]
        fighter1round['SUB. ATT'],fighter2round['SUB. ATT'] = [x.text.strip() for x in totals_per_round[7+col_index].find_all('p')]
        fighter1round['PASS'],fighter2round['PASS'] = [x.text.strip() for x in totals_per_round[8+col_index].find_all('p')]
        fighter1round['REV.'],fighter2round['REV.'] = [x.text.strip() for x in totals_per_round[9+col_index].find_all('p')]

        bout_info['Fighter1 stats']['Totals per Round'].append(dict(fighter1round))
        bout_info['Fighter2 stats']['Totals per Round'].append(dict(fighter2round))

    #Sigs
    sig_strikes = stats[2].find_all('td')
    f1,f2 = bout_info['Fighter1 stats']['Significant Strikes'], bout_info['Fighter2 stats']['Significant Strikes']

    f1['Name'],f2['Name'] = [x.text.strip() for x in totals_head[0].find_all('p')]
    f1['SIG. STR.']['landed'],f1['SIG. STR.']['attempted'], f2['SIG. STR.']['landed'],f2['SIG. STR.']['attempted'] = [a.strip() for x in sig_strikes[1].find_all('p') for a in x.text.split('of')]
    f1['SIG. STR. %'],f2['SIG. STR. %'] = [x.text.strip() for x in sig_strikes[2].find_all('p')]
    f1['HEAD']['landed'],f1['HEAD']['attempted'], f2['HEAD']['landed'],f2['HEAD']['attempted'] = [a.strip() for x in sig_strikes[3].find_all('p') for a in x.text.split('of')]
    f1['BODY']['landed'],f1['BODY']['attempted'], f2['BODY']['landed'],f2['BODY']['attempted'] = [a.strip() for x in sig_strikes[4].find_all('p') for a in x.text.split('of')]
    f1['LEG']['landed'],f1['LEG']['attempted'], f2['LEG']['landed'],f2['LEG']['attempted'] = [a.strip() for x in sig_strikes[5].find_all('p') for a in x.text.split('of')]
    f1['DISTANCE']['landed'],f1['DISTANCE']['attempted'], f2['DISTANCE']['landed'],f2['DISTANCE']['attempted'] = [a.strip() for x in sig_strikes[6].find_all('p') for a in x.text.split('of')]
    f1['CLINCH']['landed'],f1['CLINCH']['attempted'], f2['CLINCH']['landed'],f2['CLINCH']['attempted'] = [a.strip() for x in sig_strikes[7].find_all('p') for a in x.text.split('of')]
    f1['GROUND']['landed'],f1['GROUND']['attempted'], f2['GROUND']['landed'],f2['GROUND']['attempted'] = [a.strip() for x in sig_strikes[8].find_all('p') for a in x.text.split('of')]

    #SigsPR

    sig_strikes_per_round = stats[3].find_all('td')

    for col_index in range(0,len(sig_strikes_per_round),9):
        fighter1round = copy.deepcopy(sigstrikes_round)
        fighter2round = copy.deepcopy(sigstrikes_round)

        fighter1round['Round'], fighter2round['Round'] = [f'{((col_index+3)//10)+1}'] * 2
        fighter1round['Name'],fighter2round['Name'] = [x.text.strip() for x in sig_strikes_per_round[0+col_index].find_all('p')]
        fighter1round['SIG. STR.']['landed'],fighter1round['SIG. STR.']['attempted'], fighter2round['SIG. STR.']['landed'],fighter2round['SIG. STR.']['attempted'] = [a.strip() for x in sig_strikes_per_round[1+col_index].find_all('p') for a in x.text.split('of')]
        fighter1round['SIG. STR. %'],fighter2round['SIG. STR. %'] = [x.text.strip() for x in sig_strikes_per_round[2+col_index].find_all('p')]
        fighter1round['HEAD']['landed'],fighter1round['HEAD']['attempted'], fighter2round['HEAD']['landed'],fighter2round['HEAD']['attempted'] = [a.strip() for x in sig_strikes_per_round[3+col_index].find_all('p') for a in x.text.split('of')]
        fighter1round['BODY']['landed'],fighter1round['BODY']['attempted'], fighter2round['BODY']['landed'],fighter2round['BODY']['attempted'] = [a.strip() for x in sig_strikes_per_round[4+col_index].find_all('p') for a in x.text.split('of')]
        fighter1round['LEG']['landed'],fighter1round['LEG']['attempted'], fighter2round['LEG']['landed'],fighter2round['LEG']['attempted'] = [a.strip() for x in sig_strikes_per_round[5+col_index].find_all('p') for a in x.text.split('of')]
        fighter1round['DISTANCE']['landed'],fighter1round['DISTANCE']['attempted'], fighter2round['DISTANCE']['landed'],fighter2round['DISTANCE']['attempted'] = [a.strip() for x in sig_strikes_per_round[6+col_index].find_all('p') for a in x.text.split('of')]
        fighter1round['CLINCH']['landed'],fighter1round['CLINCH']['attempted'], fighter2round['CLINCH']['landed'],fighter2round['CLINCH']['attempted'] = [a.strip() for x in sig_strikes_per_round[7+col_index].find_all('p') for a in x.text.split('of')]
        fighter1round['GROUND']['landed'],fighter1round['GROUND']['attempted'], fighter2round['GROUND']['landed'],fighter2round['GROUND']['attempted'] = [a.strip() for x in sig_strikes_per_round[8+col_index].find_all('p') for a in x.text.split('of')]

        bout_info['Fighter1 stats']['Significant Strikes per Round'].append(dict(fighter1round))
        bout_info['Fighter2 stats']['Significant Strikes per Round'].append(dict(fighter2round))
    
    return bout_info

In [190]:
def get_fighters_info(url):
    site = BeautifulSoup(urlopen(url),'html.parser')
    
    fighter = {'Name':'',
               'Nickname':'',
               'Weight':'',
               'Height':'',
               'Reach':'',
               'STANCE':'',
               'DOB':'',
               'Carrer stats':{"SLpM":'', 
                               'Str. Acc.':'', 
                               'SApM':'', 
                               'Str. Def':'',  
                               'TD Avg.':'', 
                               "TD Acc.":"", 
                               "TD Def.":'', 
                               "Sub. Avg.":''},
              'bouts':[]}
    
    fighter['Name'] = site.find('h2',class_='b-content__title').span.text.strip()
    fighter['Nickname'] = site.find('p',class_='b-content__Nickname').text.strip() if site.find('p',class_='b-content__Nickname') else ''
    
    for line in site.find('ul',class_='b-list__box-list').find_all('li'):
        fighter[line.contents[1].text.strip()[:-1]] = unidecode(line.contents[2]).strip()
    if fighter['Reach'] == '--': fighter['Reach'] = ''
    
    for line in site.find('div',class_='b-list__info-box-left clearfix').find_all('li'):
        if line.find('i').text == '\n': continue
        fighter['Carrer stats'][line.contents[1].text.strip()[:-1]] = unidecode(line.contents[2]).strip()
    
    for bout in site.find('tbody',class_='b-fight-details__table-body').find_all('tr',onclick=True):
        fighter['bouts'].append(get_bout_info(bout['data-link']))
    
    
    return fighter

In [191]:
ufc_stas_update = get_last_card(2)

Anthony Smith
Alexander Gustafsson
Aleksandar Rakic
Jimi Manuwa
Makwan Amirkhani
Chris Fishgold
Christos Giagos
Damir Hadzovic
Daniel Teymur
Sung Bin Jo
Sergey Khandozhko
Rostem Akman
Lina Lansberg
Tonya Evinger
Leonardo Santos
Stevie Ray
Frank Camacho
Nick Hein
Bea Malecki
Duda Santana
Devin Clark
Darko Stosic
Joel Alvarez
Danilo Belluardo
Anthony Smith
Alexander Gustafsson
Aleksandar Rakic
Jimi Manuwa
Makwan Amirkhani
Chris Fishgold
Christos Giagos
Damir Hadzovic
Daniel Teymur
Sung Bin Jo
Sergey Khandozhko
Rostem Akman
Lina Lansberg
Tonya Evinger
Leonardo Santos
Stevie Ray
Frank Camacho
Nick Hein
Bea Malecki
Duda Santana
Devin Clark
Darko Stosic
Joel Alvarez
Danilo Belluardo
Rafael Dos Anjos
Kevin Lee
Ian Heinisch
Antonio Carlos Junior
Felicia Spencer
Megan Anderson
Vicente Luque
Derrick Krantz
Charles Oliveira
Nik Lentz
Davi Ramos
Austin Hubbard
Aspen Ladd
Sijara Eubanks
Desmond Green
Charles Jourdain
Michel Pereira
Danny Roberts
Grant Dawson
Michael Trizano
Ed Herman
Patrick Cummin

In [ ]:
#check why their returns wrong number

In [200]:
new_ufc_stats = update(fighters_ufc_stats,ufc_stas_update)

In [201]:
len(new_ufc_stats)

1934

In [202]:
json.dump(new_ufc_stats,open('all_ufc_stats_info_June_3','w'))

In [ ]:
#events keys

In [34]:
dict_of_cards_dates = json.load(open('ufc_stats_event_dates','r'))

In [68]:
dict_of_events = {}

In [75]:
def date_decoder(string):
    months = {'Jan': '01','Feb': '02','Mar': '03',
      'Apr': '04','May': '05','Jun': '06',
      'Jul': '07','Aug': '08','Sep': '09',
      'Oct': '10','Nov': '11','Dec': '12'}
    date = re.findall('\w+',string)
    return f"{date[-1]}-{months[date[0]]}-{date[1]}" 

In [69]:
def get_all_events_dates(fighters_link):
    for idx,l in enumerate(fighters_link):
        print(idx,'\t',end='')
        site = BeautifulSoup(urlopen(l),'html.parser')
        for cell in site.find('table',class_="b-fight-details__table_type_event-details").find_all("td",'l-page_align_left'):
            if cell.a:
                if re.search('event-details',cell.a['href']):
                    dict_of_events[cell.a.text.strip()] = date_decoder(cell.find_all('p')[-1].text.strip())
    return dict_of_events

In [70]:
def get_all_fighters(cards_links,fighter_links = dict()):
    for link in cards_links:
        site = BeautifulSoup(urlopen(link),'html.parser')
        for fight in site.find('tbody',class_='b-fight-details__table-body').find_all('tr',class_='b-fight-details__table-row'):
            for fighter in fight.find_all('a',class_='b-link_style_black'):
                if fighter.text.strip() not in fighter_links:
                    fighter_links[fighter.text.strip()] = fighter['href']
    
    return fighter_links

In [88]:
def get_last_card(ret_link=False): 
    link = f'http://ufcstats.com/statistics/events/completed'
    site = BeautifulSoup(urlopen(link),'html.parser')
    link = site.find('tbody').find_all('a',class_='b-link')[1]['href']
    if ret_link:
        return link
    fighters = get_all_fighters([link])
    get_all_events_dates(fighters.values())

In [77]:
get_last_card()

0 	1 	2 	3 	4 	5 	6 	7 	8 	9 	10 	11 	12 	13 	14 	15 	16 	17 	18 	19 	20 	21 	22 	23 	

In [79]:
dict_of_cards_dates.update(dict_of_events)

In [83]:
json.dump(dict_of_cards_dates,open('ufc_stats_event_dates_june_3','w'))

In [ ]:
#update women

In [85]:
women_dict = json.load(open("womens dict",'r'))

In [91]:
def is_woman(string):
    if re.search('women',string,re.IGNORECASE):
        return 1
    else: return 0

In [92]:
fighters_wclasses = dict()
def get_all_weightclasses(cards_links):
    for link in cards_links:
        site = BeautifulSoup(urlopen(link),'html.parser')
        for fight in site.find('tbody',class_='b-fight-details__table-body').find_all('tr',class_='b-fight-details__table-row'):
            info = fight.find_all('p','b-fight-details__table-text')
            wclass = info[11].text.strip()
            f1 = info[1].text.strip()
            f2 = info[2].text.strip()
            fighters_wclasses[f1] = wclass
            fighters_wclasses[f2] = wclass

In [93]:
get_all_weightclasses([get_last_card(ret_link=True)])

In [96]:
updated_women = {k:is_woman(v) for k,v in fighters_wclasses.items()}

In [97]:
women_dict.update(updated_women)

In [100]:
json.dump(women_dict,open("womens dict",'w'))